## Cryptobond Simulation

After encountering some issues engineering the smart contract, I have decided to make a simulation first, so that there is a very concise model for the contract... Also, we may know about what is good or bad, before writing unit tests!

In [4]:
import pandas as pd
from modsim import *
import datetime as dt

np.random.seed(1337)

_e = 2.718281828459045


These helper functions perform all the math and they can remain outside the class.  I wonder if it makes sense to make a smartcontract that just does this for donations...

In [2]:

def z_score(t, mu, d):
    return abs((t - mu) / d)

def Polya_tail(z):
    y = 1 - _e ** (-0.63661977236 * z ** 2)
    return (1 + sqrt(y)) / 2

def Winitzki_tail(z):
    a = -z ** 2 / 2
    b = 0.147 * a
    c = 1 - _e ** (a * (1.27323954474 + b) / (1 + b))
    return 0.5 * (1 + c ** 0.5)

def Lin_tail(z):
    y = 13.1946891451 * z / (9 - z)
    return 1 - 1 / (1 + _e ** y)

def calculate_w(t, zp):
    zn = z_score(t)
    w = zp - zn
    return w

An ideal choice computes simply and quickly.  All three of these pass that test.  They are similarly small, and can be deployed similarly in one line each.

But also, for the sake of not going below zero theoretical value, we have to choose an algorithm that consistently underestimates or overestimates...  In order to leave aside gas fees and arrive at a necessarily positive value at maturity, there must be some predictability in the approximation of payouts.

In [5]:
z_score(4, 6, 1) # 2 -- 98%
Lin_tail(2), Polya_tail(2), Winitzki_tail(2)

(0.977465404302258, 0.9800111797880668, 0.9841464769452225)

In [6]:
class Bond():
    def __init__(self, owner=0):
        today = dt.date.today()
        self.O = owner
        self.act = False
        print('Bond initialized, not active.')


    def activate(self, owner, tM, V):
        assert owner == self.O, f'Only the owner can activate.'
        self.V0 = V
        self.V = V
        self.tM = tM
        self.mu = self.tM / 2.0
        self.d = self.tM / 3.46410161514
        self.z = 4
        self.act = True
        return pd.Series(owner, owner, 0, 0, V)

    def give(self, owner, buyer, tn):
        assert owner == self.O, f'Only the owner can give.'
        assert tn != self.tp, f'Contract is on cooldown.'
        if tn > self.tM:
            x = self.V
            self.act = False
            print('Contract burned, value remitted.')
        else:
            zn = z_score(tn, self.mu, self.d)
            x = calculate_w(tn, self.z) * self.V0
            self.V -= x
            self.z = zn
            self.O = buyer
        return pd.Series(owner, buyer, tn, x, self.V)


In [47]:
def generate_trade(mode='random', start=0, end=364, accounts=1000):
    if mode == 'linear':
            t = int(start + 10)
            o = np.random.randint(0, accounts - 1)
            return (t,o)

    elif mode == 'random':
            t = int(start + (end - start) * np.random.rand())
            o = np.random.randint(0, accounts - 1)
            return (t,o)

generate_trade()

(330, 918)

In [51]:
params = Params(
    t_M = 365 * 3,
    value = 1000000,
    n = range(0,1000))
    
def run_sim(params):
    accounts = 1000
    history = pd.DataFrame(
        columns=['from', 'to', 'timestamp', 'payout', 'value'])
        
    bond = Bond()
    history[0] = bond.activate(0, params['tM'], params['V'])

    while bond.act == True:
        #trade
